In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-paradox/sample_submission.csv
/kaggle/input/ai-paradox/test/00025.wav
/kaggle/input/ai-paradox/test/00131.wav
/kaggle/input/ai-paradox/test/00094.wav
/kaggle/input/ai-paradox/test/00107.wav
/kaggle/input/ai-paradox/test/00015.wav
/kaggle/input/ai-paradox/test/00120.wav
/kaggle/input/ai-paradox/test/00061.wav
/kaggle/input/ai-paradox/test/00090.wav
/kaggle/input/ai-paradox/test/00116.wav
/kaggle/input/ai-paradox/test/00055.wav
/kaggle/input/ai-paradox/test/00024.wav
/kaggle/input/ai-paradox/test/00045.wav
/kaggle/input/ai-paradox/test/00043.wav
/kaggle/input/ai-paradox/test/00113.wav
/kaggle/input/ai-paradox/test/00097.wav
/kaggle/input/ai-paradox/test/00058.wav
/kaggle/input/ai-paradox/test/00020.wav
/kaggle/input/ai-paradox/test/00144.wav
/kaggle/input/ai-paradox/test/00099.wav
/kaggle/input/ai-paradox/test/00052.wav
/kaggle/input/ai-paradox/test/00149.wav
/kaggle/input/ai-paradox/test/00137.wav
/kaggle/input/ai-paradox/test/00103.wav
/kaggle/input/ai-paradox/test/000

In [2]:
# STEP 1: Install required packages
!pip install catboost -q

import os
import sys
import numpy as np
import pandas as pd
import torch
import torchaudio
import librosa
from pathlib import Path
from tqdm import tqdm
from scipy import stats
from scipy.stats import kurtosis, skew

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Model imports
import xgboost as xgb
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("="*60)
print("✅ STEP 1 COMPLETE: All imports successful")
print("="*60)
print(f"🔥 Using device: {device}")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"📦 Librosa version: {librosa.__version__}")

✅ STEP 1 COMPLETE: All imports successful
🔥 Using device: cuda
📦 NumPy version: 1.26.4
📦 Pandas version: 2.2.3
📦 PyTorch version: 2.6.0+cu124
📦 Librosa version: 0.11.0


In [4]:
# STEP 2 (FIXED): Install missing dependency and load PANNs

# Install torchlibrosa
print("📥 Installing torchlibrosa...")
!pip install torchlibrosa -q

# Clone PANNs repository (if not already done)
if not os.path.exists('/kaggle/working/panns_inference'):
    print("\n📥 Cloning PANNs repository...")
    !git clone https://github.com/qiuqiangkong/panns_inference.git
else:
    print("\n✓ PANNs repository already cloned")

sys.path.append('/kaggle/working/panns_inference')

# Download PANNs pretrained weights (if not already done)
if not os.path.exists('Cnn14.pth'):
    print("\n📥 Downloading PANNs CNN14 weights...")
    !wget -q https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth -O Cnn14.pth
else:
    print("\n✓ CNN14 weights already downloaded")

# Import and load PANNs
print("\n🔧 Loading PANNs model...")
from panns_inference import AudioTagging

model = AudioTagging(checkpoint_path='Cnn14.pth', device=device)

print("\n" + "="*60)
print("✅ STEP 2 COMPLETE: PANNs model loaded successfully!")
print("="*60)
print(f"📊 Model type: CNN14")
print(f"📊 Device: {device}")
print(f"📊 Embedding dimension: 2048")

📥 Installing torchlibrosa...

✓ PANNs repository already cloned

✓ CNN14 weights already downloaded

🔧 Loading PANNs model...
Checkpoint path: Cnn14.pth


--2025-10-15 10:21:48--  http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 108.177.12.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14675 (14K) [application/octet-stream]
Saving to: ‘/root/panns_data/class_labels_indices.csv’

     0K .......... ....                                       100% 68.8M=0s

2025-10-15 10:21:48 (68.8 MB/s) - ‘/root/panns_data/class_labels_indices.csv’ saved [14675/14675]



Using CPU.

✅ STEP 2 COMPLETE: PANNs model loaded successfully!
📊 Model type: CNN14
📊 Device: cuda
📊 Embedding dimension: 2048


In [6]:
# STEP 3: Define paths and load training/test file lists

print("📂 Setting up data paths...")

# Define base paths
BASE_PATH = '/kaggle/input/ai-paradox'
TRAIN_NORMAL = os.path.join(BASE_PATH, 'train/normal')
TRAIN_ABNORMAL = os.path.join(BASE_PATH, 'train/abnormal')
TEST_PATH = os.path.join(BASE_PATH, 'test')

# Verify paths exist
print("\n🔍 Verifying paths...")
for path_name, path in [('Train/Normal', TRAIN_NORMAL), 
                         ('Train/Abnormal', TRAIN_ABNORMAL), 
                         ('Test', TEST_PATH)]:
    if os.path.exists(path):
        print(f"   ✓ {path_name}: {path}")
    else:
        print(f"   ✗ {path_name}: NOT FOUND")

# Get training files
print("\n📥 Loading file lists...")
normal_files = sorted([os.path.join(TRAIN_NORMAL, f) 
                       for f in os.listdir(TRAIN_NORMAL) 
                       if f.endswith('.wav')])
abnormal_files = sorted([os.path.join(TRAIN_ABNORMAL, f) 
                         for f in os.listdir(TRAIN_ABNORMAL) 
                         if f.endswith('.wav')])

# Combine training files and create labels
train_files = normal_files + abnormal_files
train_labels = [0] * len(normal_files) + [1] * len(abnormal_files)

# Get test files
test_files = sorted([os.path.join(TEST_PATH, f) 
                     for f in os.listdir(TEST_PATH) 
                     if f.endswith('.wav')])

# Summary
print("\n" + "="*60)
print("✅ STEP 3 COMPLETE: Data paths configured")
print("="*60)
print(f"📊 Dataset Summary:")
print(f"   Normal files: {len(normal_files)}")
print(f"   Abnormal files: {len(abnormal_files)}")
print(f"   Total training: {len(train_files)}")
print(f"   Test files: {len(test_files)}")
print(f"   Class balance: {len(abnormal_files)/len(train_files)*100:.1f}% abnormal")
print(f"\n📁 Example files:")
print(f"   Normal: {os.path.basename(normal_files[0])}")
print(f"   Abnormal: {os.path.basename(abnormal_files[0])}")
print(f"   Test: {os.path.basename(test_files[0])}")

📂 Setting up data paths...

🔍 Verifying paths...
   ✓ Train/Normal: /kaggle/input/ai-paradox/train/normal
   ✓ Train/Abnormal: /kaggle/input/ai-paradox/train/abnormal
   ✓ Test: /kaggle/input/ai-paradox/test

📥 Loading file lists...

✅ STEP 3 COMPLETE: Data paths configured
📊 Dataset Summary:
   Normal files: 457
   Abnormal files: 165
   Total training: 622
   Test files: 156
   Class balance: 26.5% abnormal

📁 Example files:
   Normal: 000001.wav
   Abnormal: 000001.wav
   Test: 00001.wav


In [10]:
# STEP 4: Define all feature extraction functions

print("🔧 Defining feature extraction functions...")

# ===== Function 1: Load audio for PANNs (32kHz) =====
def load_audio_for_panns(file_path, target_sr=32000):
    """Load audio for PANNs at 32kHz"""
    try:
        audio, sr = librosa.load(file_path, sr=target_sr, mono=True)
        audio_tensor = torch.from_numpy(audio).float()
        return audio_tensor
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None


# ===== Function 2: Extract 253 advanced audio features =====
def extract_advanced_audio_features(file_path, sr=22050):
    """
    Extract 253 comprehensive audio features
    Based on YAMNet approach
    """
    try:
        # Load audio at 22050 Hz for librosa features
        y, _ = librosa.load(file_path, sr=sr, duration=5, mono=True)
        
        # Pad or truncate to consistent length (5 seconds)
        max_len = sr * 5
        if len(y) < max_len:
            y = np.pad(y, (0, max_len - len(y)), 'constant')
        else:
            y = y[:max_len]
        
        features = []
        
        # 1. Statistical features (9)
        features.extend([
            np.mean(y), np.std(y), np.max(y), np.min(y),
            np.median(y), kurtosis(y), skew(y),
            np.percentile(y, 25), np.percentile(y, 75)
        ])
        
        # 2. Spectral features (30)
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        features.extend([np.mean(spectral_centroids), np.std(spectral_centroids)])
        
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        features.extend([np.mean(spectral_rolloff), np.std(spectral_rolloff)])
        
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        features.extend([np.mean(spectral_bandwidth), np.std(spectral_bandwidth)])
        
        spectral_flatness = librosa.feature.spectral_flatness(y=y)[0]
        features.extend([np.mean(spectral_flatness), np.std(spectral_flatness)])
        
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        features.extend(np.mean(spectral_contrast, axis=1))
        features.extend(np.std(spectral_contrast, axis=1))
        
        rolloff_85 = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85)[0]
        rolloff_95 = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.95)[0]
        features.extend([np.mean(rolloff_85), np.mean(rolloff_95)])
        
        # 3. MFCC features (120)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)
        
        for mfcc_matrix in [mfccs, mfcc_delta, mfcc_delta2]:
            features.extend(np.mean(mfcc_matrix, axis=1))
            features.extend(np.std(mfcc_matrix, axis=1))
        
        # 4. Temporal features (6)
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        features.extend([np.mean(zcr), np.std(zcr), np.max(zcr)])
        
        rms = librosa.feature.rms(y=y)[0]
        features.extend([np.mean(rms), np.std(rms), np.max(rms)])
        
        # 5. Chroma features (72)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        features.extend(np.mean(chroma_stft, axis=1))
        features.extend(np.std(chroma_stft, axis=1))
        
        chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
        features.extend(np.mean(chroma_cqt, axis=1))
        features.extend(np.std(chroma_cqt, axis=1))
        
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
        features.extend(np.mean(chroma_cens, axis=1))
        features.extend(np.std(chroma_cens, axis=1))
        
        # 6. Mel-spectrogram (4)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        features.extend([
            np.mean(mel_spec_db), np.std(mel_spec_db),
            np.max(mel_spec_db), np.min(mel_spec_db)
        ])
        
        # 7. Tonnetz (12)
        harmonic = librosa.effects.harmonic(y)
        tonnetz = librosa.feature.tonnetz(y=harmonic, sr=sr)
        features.extend(np.mean(tonnetz, axis=1))
        features.extend(np.std(tonnetz, axis=1))
        
        # 8. Polynomial features (4)
        poly_features = librosa.feature.poly_features(y=y, sr=sr)
        features.extend(np.mean(poly_features, axis=1))
        features.extend(np.std(poly_features, axis=1))
        
        # 9. Tempogram (2)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        tempogram = librosa.feature.tempogram(onset_envelope=onset_env, sr=sr)
        features.extend([np.mean(tempogram), np.std(tempogram)])
        
        return np.array(features)
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


# ===== Function 3: Extract combined features (PANNs + Audio) =====
def extract_combined_features(audio_files, labels=None, batch_size=16):
    """
    Extract PANNs embeddings (2048) + Audio features (253)
    Total: 2301 features per file
    """
    panns_embeddings = []
    audio_features = []
    valid_labels = []
    valid_files = []
    skipped = []
    
    print(f"🎵 Extracting combined features from {len(audio_files)} files...")
    
    for i in tqdm(range(0, len(audio_files), batch_size)):
        batch_files = audio_files[i:i+batch_size]
        batch_labels = labels[i:i+batch_size] if labels is not None else [None]*len(batch_files)
        
        for file_path, label in zip(batch_files, batch_labels):
            # Extract PANNs embedding
            audio = load_audio_for_panns(file_path)
            if audio is None:
                skipped.append(file_path)
                continue
            
            audio_tensor = audio.unsqueeze(0).to(device)
            with torch.no_grad():
                output = model.inference(audio_tensor)
                panns_emb = output[1] if isinstance(output, tuple) else output
                if isinstance(panns_emb, torch.Tensor):
                    panns_emb = panns_emb.cpu().numpy()
                panns_emb = np.squeeze(panns_emb)
            
            # Extract audio features
            audio_feat = extract_advanced_audio_features(file_path)
            if audio_feat is None:
                skipped.append(file_path)
                continue
            
            panns_embeddings.append(panns_emb)
            audio_features.append(audio_feat)
            valid_files.append(os.path.basename(file_path))
            if label is not None:
                valid_labels.append(label)
    
    panns_embeddings = np.array(panns_embeddings)
    audio_features = np.array(audio_features)
    combined = np.hstack([panns_embeddings, audio_features])
    
    if skipped:
        print(f"⚠️ Skipped {len(skipped)} files")
    
    print(f"✅ PANNs embeddings: {panns_embeddings.shape}")
    print(f"✅ Audio features: {audio_features.shape}")
    print(f"✅ Combined features: {combined.shape}")
    
    return combined, valid_labels if labels is not None else None, valid_files


print("\n" + "="*60)
print("✅ STEP 4 COMPLETE: Feature extraction functions defined")
print("="*60)
print("📊 Functions ready:")
print("   • load_audio_for_panns() - Loads audio at 32kHz for PANNs")
print("   • extract_advanced_audio_features() - Extracts 253 features")
print("   • extract_combined_features() - Combines PANNs + audio features")

🔧 Defining feature extraction functions...

✅ STEP 4 COMPLETE: Feature extraction functions defined
📊 Functions ready:
   • load_audio_for_panns() - Loads audio at 32kHz for PANNs
   • extract_advanced_audio_features() - Extracts 253 features
   • extract_combined_features() - Combines PANNs + audio features


In [14]:
# STEP 5: Define preprocessing functions

print("🔧 Defining preprocessing functions...")

# ===== Custom SMOTE Implementation =====
class SimpleSMOTE:
    """Custom SMOTE - handles class imbalance"""
    def __init__(self, k_neighbors=3, random_state=42):
        self.k_neighbors = k_neighbors
        self.random_state = random_state
        
    def fit_resample(self, X, y):
        np.random.seed(self.random_state)
        
        # Separate majority and minority classes
        X_maj = X[y == 0]
        X_min = X[y == 1]
        y_maj = y[y == 0]
        y_min = y[y == 1]
        
        # Calculate how many synthetic samples needed
        n_samples_needed = len(X_maj) - len(X_min)
        
        if n_samples_needed <= 0:
            return X, y
        
        # Fit nearest neighbors on minority class
        k = min(self.k_neighbors + 1, len(X_min))
        nn = NearestNeighbors(n_neighbors=k)
        nn.fit(X_min)
        
        # Generate synthetic samples
        synthetic_samples = []
        for _ in range(n_samples_needed):
            idx = np.random.randint(0, len(X_min))
            sample = X_min[idx]
            neighbors_idx = nn.kneighbors([sample], return_distance=False)[0][1:]
            
            if len(neighbors_idx) > 0:
                neighbor_idx = np.random.choice(neighbors_idx)
                neighbor = X_min[neighbor_idx]
                alpha = np.random.random()
                synthetic = sample + alpha * (neighbor - sample)
                synthetic_samples.append(synthetic)
        
        # Combine all samples
        if len(synthetic_samples) > 0:
            X_synthetic = np.array(synthetic_samples)
            y_synthetic = np.ones(len(X_synthetic), dtype=int)
            X_balanced = np.vstack([X_maj, X_min, X_synthetic])
            y_balanced = np.hstack([y_maj, y_min, y_synthetic])
        else:
            X_balanced = np.vstack([X_maj, X_min])
            y_balanced = np.hstack([y_maj, y_min])
        
        # Shuffle
        shuffle_idx = np.random.permutation(len(X_balanced))
        return X_balanced[shuffle_idx], y_balanced[shuffle_idx]


# ===== Preprocessing Pipeline =====
def preprocess_features(X_train, y_train, X_val=None):
    """
    Complete preprocessing pipeline:
    1. Clean data (NaN, inf)
    2. Remove low-variance features
    3. Robust scaling
    4. SMOTE balancing
    """
    print("\n🔧 PREPROCESSING PIPELINE")
    print("="*60)
    
    # Step 1: Clean data
    print("   Step 1: Cleaning data...")
    X_train = np.nan_to_num(X_train)
    X_train = np.clip(X_train, -1e10, 1e10)
    print(f"      ✓ Removed NaN/Inf values")
    
    # Step 2: Variance threshold
    print("   Step 2: Removing low-variance features...")
    var_thresh = VarianceThreshold(threshold=0.001)
    X_train_filtered = var_thresh.fit_transform(X_train)
    removed = X_train.shape[1] - X_train_filtered.shape[1]
    print(f"      ✓ Removed {removed} low-variance features")
    print(f"      ✓ Remaining: {X_train_filtered.shape[1]} features")
    
    # Step 3: Robust scaling
    print("   Step 3: Scaling features...")
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train_filtered)
    print(f"      ✓ Features scaled with RobustScaler")
    
    # Step 4: SMOTE
    print("   Step 4: Balancing classes with SMOTE...")
    print(f"      Before: Normal={np.sum(y_train==0)}, Abnormal={np.sum(y_train==1)}")
    smote = SimpleSMOTE(k_neighbors=3, random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
    print(f"      After:  Normal={np.sum(y_train_balanced==0)}, Abnormal={np.sum(y_train_balanced==1)}")
    print(f"      ✓ Generated {len(y_train_balanced) - len(y_train)} synthetic samples")
    
    # Process validation set if provided
    if X_val is not None:
        X_val = np.nan_to_num(X_val)
        X_val = np.clip(X_val, -1e10, 1e10)
        X_val_filtered = var_thresh.transform(X_val)
        X_val_scaled = scaler.transform(X_val_filtered)
        print(f"\n   ✓ Validation set preprocessed: {X_val_scaled.shape}")
        return X_train_balanced, y_train_balanced, X_val_scaled, var_thresh, scaler
    
    return X_train_balanced, y_train_balanced, None, var_thresh, scaler


print("\n" + "="*60)
print("✅ STEP 5 COMPLETE: Preprocessing functions defined")
print("="*60)
print("📊 Components ready:")
print("   • SimpleSMOTE class - Synthetic oversampling")
print("   • preprocess_features() - Complete preprocessing pipeline")
print("   • VarianceThreshold - Removes useless features")
print("   • RobustScaler - Handles outliers")

🔧 Defining preprocessing functions...

✅ STEP 5 COMPLETE: Preprocessing functions defined
📊 Components ready:
   • SimpleSMOTE class - Synthetic oversampling
   • preprocess_features() - Complete preprocessing pipeline
   • VarianceThreshold - Removes useless features
   • RobustScaler - Handles outliers


In [17]:
# STEP 6: Define model training and ensemble functions

print("🔧 Defining model training functions...")


# ===== Train 4 Optimized Models =====
def train_optimized_models(X_train, y_train, X_val, y_val):
    """
    Train 4 models: CatBoost, XGBoost, RandomForest, ExtraTrees
    """
    models = {}
    scores = {}
    
    # Calculate class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    scale_pos_weight = class_weights[1] / class_weights[0]
    
    print("\n🚀 TRAINING OPTIMIZED MODELS")
    print("="*60)
    print(f"   Class weight ratio: {scale_pos_weight:.2f}")
    
    # 1. CatBoost
    print("\n1️⃣ Training CatBoost...")
    cat_model = CatBoostClassifier(
        iterations=2000,
        learning_rate=0.05,
        depth=10,
        l2_leaf_reg=5,
        random_seed=42,
        verbose=0,
        early_stopping_rounds=100,
        class_weights=[1.0, 2.0]
    )
    cat_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    
    y_pred_cat = cat_model.predict(X_val)
    y_proba_cat = cat_model.predict_proba(X_val)[:, 1]
    auc_cat = roc_auc_score(y_val, y_proba_cat)
    acc_cat = accuracy_score(y_val, y_pred_cat)
    
    models['CatBoost'] = cat_model
    scores['CatBoost'] = {'auc': auc_cat, 'acc': acc_cat, 'proba': y_proba_cat}
    print(f"   ✅ AUC: {auc_cat:.4f} | Accuracy: {acc_cat:.4f}")
    
    # 2. XGBoost
    print("\n2️⃣ Training XGBoost...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.05,
        reg_alpha=0.2,
        reg_lambda=0.2,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight=2.0,
        eval_metric='logloss',
        early_stopping_rounds=100
    )
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    y_pred_xgb = xgb_model.predict(X_val)
    y_proba_xgb = xgb_model.predict_proba(X_val)[:, 1]
    auc_xgb = roc_auc_score(y_val, y_proba_xgb)
    acc_xgb = accuracy_score(y_val, y_pred_xgb)
    
    models['XGBoost'] = xgb_model
    scores['XGBoost'] = {'auc': auc_xgb, 'acc': acc_xgb, 'proba': y_proba_xgb}
    print(f"   ✅ AUC: {auc_xgb:.4f} | Accuracy: {acc_xgb:.4f}")
    
    # 3. Random Forest
    print("\n3️⃣ Training Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=500,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    rf_model.fit(X_train, y_train)
    
    y_pred_rf = rf_model.predict(X_val)
    y_proba_rf = rf_model.predict_proba(X_val)[:, 1]
    auc_rf = roc_auc_score(y_val, y_proba_rf)
    acc_rf = accuracy_score(y_val, y_pred_rf)
    
    models['RandomForest'] = rf_model
    scores['RandomForest'] = {'auc': auc_rf, 'acc': acc_rf, 'proba': y_proba_rf}
    print(f"   ✅ AUC: {auc_rf:.4f} | Accuracy: {acc_rf:.4f}")
    
    # 4. Extra Trees
    print("\n4️⃣ Training Extra Trees...")
    et_model = ExtraTreesClassifier(
        n_estimators=500,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    et_model.fit(X_train, y_train)
    
    y_pred_et = et_model.predict(X_val)
    y_proba_et = et_model.predict_proba(X_val)[:, 1]
    auc_et = roc_auc_score(y_val, y_proba_et)
    acc_et = accuracy_score(y_val, y_pred_et)
    
    models['ExtraTrees'] = et_model
    scores['ExtraTrees'] = {'auc': auc_et, 'acc': acc_et, 'proba': y_proba_et}
    print(f"   ✅ AUC: {auc_et:.4f} | Accuracy: {acc_et:.4f}")
    
    return models, scores


# ===== Create Weighted Ensemble with Threshold Optimization =====
def create_optimized_ensemble(models, scores, X_val, y_val):
    """
    Create weighted ensemble using F1 scores
    Optimize threshold for best F1
    """
    print("\n🔮 CREATING WEIGHTED ENSEMBLE")
    print("="*60)
    
    model_weights = {}
    
    # Calculate F1-based weights
    print("\n📊 Calculating model weights based on F1 scores...")
    for name in models.keys():
        best_f1 = 0
        for threshold in np.arange(0.2, 0.8, 0.01):
            y_pred = (scores[name]['proba'] > threshold).astype(int)
            f1 = f1_score(y_val, y_pred)
            if f1 > best_f1:
                best_f1 = f1
        
        model_weights[name] = best_f1
        print(f"   {name}: F1 = {best_f1:.4f}")
    
    # Normalize weights
    total_weight = sum(model_weights.values())
    for name in model_weights:
        model_weights[name] /= total_weight
    
    print(f"\n📊 Normalized model weights:")
    for name, weight in model_weights.items():
        print(f"   {name}: {weight:.4f}")
    
    # Create ensemble predictions
    ensemble_proba = np.zeros(len(y_val))
    for name, weight in model_weights.items():
        ensemble_proba += weight * scores[name]['proba']
    
    # Optimize threshold
    print(f"\n🎯 Optimizing threshold...")
    best_threshold = 0.5
    best_f1 = 0
    
    for threshold in np.arange(0.2, 0.8, 0.005):
        y_pred = (ensemble_proba > threshold).astype(int)
        f1 = f1_score(y_val, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    
    ensemble_pred = (ensemble_proba >= best_threshold).astype(int)
    auc_ensemble = roc_auc_score(y_val, ensemble_proba)
    acc_ensemble = accuracy_score(y_val, ensemble_pred)
    
    print(f"\n✅ Ensemble Performance:")
    print(f"   Optimal Threshold: {best_threshold:.3f}")
    print(f"   Best F1 Score: {best_f1:.4f}")
    print(f"   AUC-ROC: {auc_ensemble:.4f}")
    print(f"   Accuracy: {acc_ensemble:.4f}")
    
    print(f"\n📋 Classification Report:")
    print(classification_report(y_val, ensemble_pred, target_names=['Normal', 'Abnormal']))
    
    return model_weights, best_threshold, best_f1


print("\n" + "="*60)
print("✅ STEP 6 COMPLETE: Model training functions defined")
print("="*60)
print("📊 Functions ready:")
print("   • train_optimized_models() - Trains 4 models with early stopping")
print("   • create_optimized_ensemble() - F1-weighted ensemble with threshold opt")

🔧 Defining model training functions...

✅ STEP 6 COMPLETE: Model training functions defined
📊 Functions ready:
   • train_optimized_models() - Trains 4 models with early stopping
   • create_optimized_ensemble() - F1-weighted ensemble with threshold opt


In [18]:
# STEP 7: Extract combined features from training data

print("\n" + "="*60)
print("🎬 STARTING FEATURE EXTRACTION - TRAINING DATA")
print("="*60)
print(f"📊 Files to process: {len(train_files)}")
print(f"⏱️  Estimated time: 10-15 minutes")
print(f"🔧 Batch size: 16")
print("\n🚀 Starting extraction...\n")

# Extract features
train_features, train_labels_valid, _ = extract_combined_features(
    train_files,
    train_labels,
    batch_size=16
)

# Convert labels to numpy array
y_train_full = np.array(train_labels_valid)

print("\n" + "="*60)
print("✅ STEP 7 COMPLETE: Training features extracted")
print("="*60)
print(f"📊 Feature matrix shape: {train_features.shape}")
print(f"📊 Labels shape: {y_train_full.shape}")
print(f"📊 Features per sample: {train_features.shape[1]}")
print(f"   - PANNs embeddings: 2048")
print(f"   - Audio features: {train_features.shape[1] - 2048}")
print(f"\n📊 Label distribution:")
print(f"   Normal (0): {np.sum(y_train_full==0)} samples")
print(f"   Abnormal (1): {np.sum(y_train_full==1)} samples")


🎬 STARTING FEATURE EXTRACTION - TRAINING DATA
📊 Files to process: 622
⏱️  Estimated time: 10-15 minutes
🔧 Batch size: 16

🚀 Starting extraction...

🎵 Extracting combined features from 622 files...


100%|██████████| 39/39 [12:01<00:00, 18.51s/it]

✅ PANNs embeddings: (622, 2048)
✅ Audio features: (622, 253)
✅ Combined features: (622, 2301)

✅ STEP 7 COMPLETE: Training features extracted
📊 Feature matrix shape: (622, 2301)
📊 Labels shape: (622,)
📊 Features per sample: 2301
   - PANNs embeddings: 2048
   - Audio features: 253

📊 Label distribution:
   Normal (0): 457 samples
   Abnormal (1): 165 samples


In [19]:
# STEP 8: Train/Validation split and preprocessing

print("\n" + "="*60)
print("📊 STEP 8: TRAIN/VALIDATION SPLIT & PREPROCESSING")
print("="*60)

# Split into train and validation (85/15 like YAMNet approach)
print("\n🔀 Splitting data (85% train, 15% validation)...")
X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    train_features,
    y_train_full,
    test_size=0.15,
    random_state=42,
    stratify=y_train_full
)

print(f"   Train set: {X_train_raw.shape}")
print(f"   Val set: {X_val_raw.shape}")
print(f"   Train labels: Normal={np.sum(y_train==0)}, Abnormal={np.sum(y_train==1)}")
print(f"   Val labels: Normal={np.sum(y_val==0)}, Abnormal={np.sum(y_val==1)}")

# Apply preprocessing pipeline
X_train, y_train_balanced, X_val, var_thresh, scaler = preprocess_features(
    X_train_raw, y_train, X_val_raw
)

print("\n" + "="*60)
print("✅ STEP 8 COMPLETE: Data split and preprocessed")
print("="*60)
print(f"📊 Final shapes:")
print(f"   Train (balanced): {X_train.shape}")
print(f"   Train labels (balanced): {y_train_balanced.shape}")
print(f"   Validation: {X_val.shape}")
print(f"   Validation labels: {y_val.shape}")


📊 STEP 8: TRAIN/VALIDATION SPLIT & PREPROCESSING

🔀 Splitting data (85% train, 15% validation)...
   Train set: (528, 2301)
   Val set: (94, 2301)
   Train labels: Normal=388, Abnormal=140
   Val labels: Normal=69, Abnormal=25

🔧 PREPROCESSING PIPELINE
   Step 1: Cleaning data...
      ✓ Removed NaN/Inf values
   Step 2: Removing low-variance features...
      ✓ Removed 1637 low-variance features
      ✓ Remaining: 664 features
   Step 3: Scaling features...
      ✓ Features scaled with RobustScaler
   Step 4: Balancing classes with SMOTE...
      Before: Normal=388, Abnormal=140
      After:  Normal=388, Abnormal=388
      ✓ Generated 248 synthetic samples

   ✓ Validation set preprocessed: (94, 664)

✅ STEP 8 COMPLETE: Data split and preprocessed
📊 Final shapes:
   Train (balanced): (776, 664)
   Train labels (balanced): (776,)
   Validation: (94, 664)
   Validation labels: (94,)


In [20]:
# STEP 9: Train all 4 optimized models

print("\n" + "="*60)
print("🚀 STEP 9: TRAINING ALL MODELS")
print("="*60)
print(f"📊 Training on {X_train.shape[0]} samples")
print(f"📊 Validating on {X_val.shape[0]} samples")
print(f"⏱️  Estimated time: 5-10 minutes")
print("\n🎯 Starting training...\n")

# Train all 4 models
models, scores = train_optimized_models(X_train, y_train_balanced, X_val, y_val)

print("\n" + "="*60)
print("✅ STEP 9 COMPLETE: All models trained")
print("="*60)
print("\n📊 Model Performance Summary:")
print("-" * 60)
for name in models.keys():
    print(f"{name:15} | AUC: {scores[name]['auc']:.4f} | Acc: {scores[name]['acc']:.4f}")
print("-" * 60)

# Find best individual model
best_model_name = max(scores.keys(), key=lambda x: scores[x]['auc'])
best_auc = scores[best_model_name]['auc']
print(f"\n🏆 Best individual model: {best_model_name} (AUC: {best_auc:.4f})")


🚀 STEP 9: TRAINING ALL MODELS
📊 Training on 776 samples
📊 Validating on 94 samples
⏱️  Estimated time: 5-10 minutes

🎯 Starting training...


🚀 TRAINING OPTIMIZED MODELS
   Class weight ratio: 1.00

1️⃣ Training CatBoost...
   ✅ AUC: 0.9965 | Accuracy: 0.9681

2️⃣ Training XGBoost...
   ✅ AUC: 0.9901 | Accuracy: 0.9574

3️⃣ Training Random Forest...
   ✅ AUC: 0.9861 | Accuracy: 0.9362

4️⃣ Training Extra Trees...
   ✅ AUC: 0.9768 | Accuracy: 0.8936

✅ STEP 9 COMPLETE: All models trained

📊 Model Performance Summary:
------------------------------------------------------------
CatBoost        | AUC: 0.9965 | Acc: 0.9681
XGBoost         | AUC: 0.9901 | Acc: 0.9574
RandomForest    | AUC: 0.9861 | Acc: 0.9362
ExtraTrees      | AUC: 0.9768 | Acc: 0.8936
------------------------------------------------------------

🏆 Best individual model: CatBoost (AUC: 0.9965)


In [21]:
# STEP 10: Create weighted ensemble with threshold optimization

print("\n" + "="*60)
print("🔮 STEP 10: CREATING WEIGHTED ENSEMBLE")
print("="*60)
print("🎯 Combining all 4 models with F1-weighted voting...")
print("🎯 Optimizing classification threshold...\n")

# Create ensemble
model_weights, best_threshold, best_f1 = create_optimized_ensemble(
    models, scores, X_val, y_val
)

print("\n" + "="*60)
print("✅ STEP 10 COMPLETE: Ensemble created and optimized")
print("="*60)

# Compare individual vs ensemble
print("\n📊 FINAL COMPARISON:")
print("="*60)
print("Individual Models:")
for name in models.keys():
    print(f"  {name:15} | AUC: {scores[name]['auc']:.4f}")
print("\n🔮 Weighted Ensemble:")
ensemble_proba = np.zeros(len(y_val))
for name, weight in model_weights.items():
    ensemble_proba += weight * scores[name]['proba']
ensemble_auc = roc_auc_score(y_val, ensemble_proba)
print(f"  Ensemble        | AUC: {ensemble_auc:.4f}")
print("="*60)

if ensemble_auc > max([scores[name]['auc'] for name in scores.keys()]):
    improvement = ensemble_auc - max([scores[name]['auc'] for name in scores.keys()])
    print(f"\n🎉 Ensemble improved by {improvement:.4f} points!")
else:
    print(f"\n💡 Best individual model (CatBoost) still leads!")


🔮 STEP 10: CREATING WEIGHTED ENSEMBLE
🎯 Combining all 4 models with F1-weighted voting...
🎯 Optimizing classification threshold...


🔮 CREATING WEIGHTED ENSEMBLE

📊 Calculating model weights based on F1 scores...
   CatBoost: F1 = 0.9804
   XGBoost: F1 = 0.9200
   RandomForest: F1 = 0.9057
   ExtraTrees: F1 = 0.8750

📊 Normalized model weights:
   CatBoost: 0.2663
   XGBoost: 0.2499
   RandomForest: 0.2460
   ExtraTrees: 0.2377

🎯 Optimizing threshold...

✅ Ensemble Performance:
   Optimal Threshold: 0.350
   Best F1 Score: 0.9434
   AUC-ROC: 0.9925
   Accuracy: 0.9681

📋 Classification Report:
              precision    recall  f1-score   support

      Normal       1.00      0.96      0.98        69
    Abnormal       0.89      1.00      0.94        25

    accuracy                           0.97        94
   macro avg       0.95      0.98      0.96        94
weighted avg       0.97      0.97      0.97        94


✅ STEP 10 COMPLETE: Ensemble created and optimized

📊 FINAL COMPARISO

In [22]:
# STEP 11: Retrain on full data and predict test set

print("\n" + "="*60)
print("🚀 STEP 11: FINAL TRAINING & TEST PREDICTIONS")
print("="*60)

# Preprocess ALL training data (no split)
print("\n🔧 Preprocessing full training dataset...")
X_all_balanced, y_all_balanced, _, var_thresh_final, scaler_final = preprocess_features(
    train_features, y_train_full
)

print(f"   Final training size: {X_all_balanced.shape}")

# Retrain CatBoost (the winner) on full dataset
print("\n🏆 Retraining CatBoost on full balanced dataset...")
final_model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=10,
    l2_leaf_reg=5,
    random_seed=42,
    verbose=0,
    class_weights=[1.0, 2.0]
)
final_model.fit(X_all_balanced, y_all_balanced)
print("   ✅ Final model trained!")

# Extract test features
print("\n🎵 Extracting features from test set...")
print(f"   Files to process: {len(test_files)}")
print(f"   Estimated time: 2-3 minutes\n")

test_features, _, test_filenames = extract_combined_features(
    test_files,
    labels=None,
    batch_size=16
)

# Preprocess test data
print("\n🔧 Preprocessing test data...")
test_features_clean = np.nan_to_num(test_features)
test_features_clean = np.clip(test_features_clean, -1e10, 1e10)
test_features_filtered = var_thresh_final.transform(test_features_clean)
test_features_scaled = scaler_final.transform(test_features_filtered)
print(f"   Test features preprocessed: {test_features_scaled.shape}")

# Generate predictions
print("\n🎯 Generating predictions...")
test_proba = final_model.predict_proba(test_features_scaled)[:, 1]
test_predictions = (test_proba >= best_threshold).astype(int)

print("\n" + "="*60)
print("✅ STEP 11 COMPLETE: Test predictions generated")
print("="*60)
print(f"📊 Test Predictions:")
print(f"   Normal (0): {np.sum(test_predictions==0)} files ({np.sum(test_predictions==0)/len(test_predictions)*100:.1f}%)")
print(f"   Abnormal (1): {np.sum(test_predictions==1)} files ({np.sum(test_predictions==1)/len(test_predictions)*100:.1f}%)")
print(f"   Threshold used: {best_threshold:.3f}")


🚀 STEP 11: FINAL TRAINING & TEST PREDICTIONS

🔧 Preprocessing full training dataset...

🔧 PREPROCESSING PIPELINE
   Step 1: Cleaning data...
      ✓ Removed NaN/Inf values
   Step 2: Removing low-variance features...
      ✓ Removed 1638 low-variance features
      ✓ Remaining: 663 features
   Step 3: Scaling features...
      ✓ Features scaled with RobustScaler
   Step 4: Balancing classes with SMOTE...
      Before: Normal=457, Abnormal=165
      After:  Normal=457, Abnormal=457
      ✓ Generated 292 synthetic samples
   Final training size: (914, 663)

🏆 Retraining CatBoost on full balanced dataset...
   ✅ Final model trained!

🎵 Extracting features from test set...
   Files to process: 156
   Estimated time: 2-3 minutes

🎵 Extracting combined features from 156 files...


100%|██████████| 10/10 [03:05<00:00, 18.53s/it]

✅ PANNs embeddings: (156, 2048)
✅ Audio features: (156, 253)
✅ Combined features: (156, 2301)

🔧 Preprocessing test data...
   Test features preprocessed: (156, 663)

🎯 Generating predictions...

✅ STEP 11 COMPLETE: Test predictions generated
📊 Test Predictions:
   Normal (0): 115 files (73.7%)
   Abnormal (1): 41 files (26.3%)
   Threshold used: 0.350


In [23]:
# STEP 12: Create final submission CSV

print("\n" + "="*60)
print("📝 STEP 12: CREATING SUBMISSION FILE")
print("="*60)

# Create submission dataframe
submission_df = pd.DataFrame({
    'file_name': test_filenames,
    'target': test_predictions
})

# Sort by filename (important for competition)
submission_df = submission_df.sort_values('file_name').reset_index(drop=True)

# Save submission
submission_path = 'panns_advanced.csv'
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ Submission file created: {submission_path}")
print(f"\n📄 First 10 predictions:")
print(submission_df.head(10).to_string(index=False))
print(f"\n📄 Last 10 predictions:")
print(submission_df.tail(10).to_string(index=False))

print("\n" + "="*60)
print("📊 FINAL SUBMISSION STATISTICS")
print("="*60)
print(f"Total files: {len(submission_df)}")
print(f"Normal predictions: {np.sum(submission_df['target']==0)} ({np.sum(submission_df['target']==0)/len(submission_df)*100:.1f}%)")
print(f"Abnormal predictions: {np.sum(submission_df['target']==1)} ({np.sum(submission_df['target']==1)/len(submission_df)*100:.1f}%)")

print("\n" + "="*60)
print("🎊 ALL STEPS COMPLETE!")
print("="*60)
print("\n🏆 FINAL MODEL PERFORMANCE:")
print(f"   Model: CatBoost")
print(f"   Validation AUC-ROC: 0.9965 (99.65%)")
print(f"   Validation Accuracy: 96.81%")
print(f"   Validation F1-Score: 0.9804")
print(f"   Features used: 2301 (PANNs 2048 + Audio 253)")
print(f"   Preprocessing: VarianceThreshold + RobustScaler + SMOTE")
print(f"   Class weights: [1.0, 2.0]")
print(f"   Optimal threshold: {best_threshold:.3f}")

print("\n📥 Download your submission file and submit to Kaggle!")
print("="*60)


📝 STEP 12: CREATING SUBMISSION FILE

✅ Submission file created: panns_advanced.csv

📄 First 10 predictions:
file_name  target
00001.wav       0
00002.wav       0
00003.wav       0
00004.wav       0
00005.wav       0
00006.wav       0
00007.wav       0
00008.wav       0
00009.wav       0
00010.wav       0

📄 Last 10 predictions:
file_name  target
00147.wav       1
00148.wav       1
00149.wav       1
00150.wav       1
00151.wav       1
00152.wav       1
00153.wav       1
00154.wav       1
00155.wav       1
00156.wav       1

📊 FINAL SUBMISSION STATISTICS
Total files: 156
Normal predictions: 115 (73.7%)
Abnormal predictions: 41 (26.3%)

🎊 ALL STEPS COMPLETE!

🏆 FINAL MODEL PERFORMANCE:
   Model: CatBoost
   Validation AUC-ROC: 0.9965 (99.65%)
   Validation Accuracy: 96.81%
   Validation F1-Score: 0.9804
   Features used: 2301 (PANNs 2048 + Audio 253)
   Preprocessing: VarianceThreshold + RobustScaler + SMOTE
   Class weights: [1.0, 2.0]
   Optimal threshold: 0.350

📥 Download your submis